In [ ]:
# Animal Image Classifier (Animals-10)

This project trains a **ResNet50** deep learning model on the [Animals-10 dataset](https://www.kaggle.com/datasets/alessiocorrado99/animals10) to classify photos of 10 different animals:
Dog, Cat, Elephant, Cow, Horse, Sheep, Chicken, Spider, Squirrel, Butterfly.

The notebook uses **PyTorch** for training and **Gradio** for an interactive web demo.


In [ ]:
# Core libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

# GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Using device:", device)


In [ ]:
# Mount Kaggle and download the dataset (if not already done)
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()  # upload your kaggle.json manually

!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d alessiocorrado99/animals10
!unzip -q animals10.zip -d animals_dataset


In [ ]:
cd animals_dataset/raw-img

mv cane dog
mv cavallo horse
mv elefante elephant
mv farfalla butterfly
mv gallina chicken
mv gatto cat
mv mucca cow
mv pecora sheep
mv ragno spider
mv scoiattolo squirrel

cd /content


In [ ]:
# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
dataset = datasets.ImageFolder("animals_dataset/raw-img", transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=32, num_workers=2)

print("Classes:", dataset.classes)
print(f"Training samples: {len(train_data)} | Testing samples: {len(test_data)}")


In [ ]:
# Load pretrained ResNet50
model = models.resnet50(weights="IMAGENET1K_V1")

# Adjust the final fully connected layer for 10 classes
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print("Model ready for training")


In [ ]:
epochs = 5
train_losses, train_accuracies = [], []

for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")


In [ ]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")


In [ ]:
torch.save(model.state_dict(), "animal_classifier.pth")
print("Model saved as animal_classifier.pth")


In [ ]:
import gradio as gr
from PIL import Image
import torch.nn.functional as F

def classify_image(img):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(x)
        probs = F.softmax(outputs, dim=1)[0]
    return {dataset.classes[i]: float(probs[i]) for i in range(len(probs))}

gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="🐾 Animal Classifier",
    description="Upload a photo of an animal (dog, cat, elephant, etc.) to see what the model predicts!"
).launch(share=True)
